# How to add message history

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Chaining runnables](/docs/how_to/sequence/)
- [Configuring chain parameters at runtime](/docs/how_to/configure)
- [Prompt templates](/docs/concepts/#prompt-templates)
- [Chat Messages](/docs/concepts/#message-types)

:::

Passing conversation state into and out a chain is vital when building a chatbot. The [`RunnableWithMessageHistory`](https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html#langchain_core.runnables.history.RunnableWithMessageHistory) class lets us add message history to certain types of chains. It wraps another Runnable and manages the chat message history for it. Specifically, it loads previous messages in the conversation BEFORE passing it to the Runnable, and it saves the generated response as a message AFTER calling the runnable. This class also enables multiple conversations by saving each conversation with a `session_id` - it then expects a `session_id` to be passed in the config when calling the runnable, and uses that to look up the relevant conversation history.

![index_diagram](../../static/img/message_history.png)

In practice this looks something like:

```python
from langchain_core.runnables.history import RunnableWithMessageHistory


with_message_history = RunnableWithMessageHistory(
    # The underlying runnable
    runnable,  
    # A function that takes in a session id and returns a memory object
    get_session_history,  
    # Other parameters that may be needed to align the inputs/outputs
    # of the Runnable with the memory object
    ...  
)

with_message_history.invoke(
    # The same input as before
    {"ability": "math", "input": "What does cosine mean?"},
    # Configuration specifying the `session_id`,
    # which controls which conversation to load
    config={"configurable": {"session_id": "abc123"}},
)
```


In order to properly set this up there are two main things to consider:

1. How to store and load messages? (this is `get_session_history` in the example above)
2. What is the underlying Runnable you are wrapping and what are its inputs/outputs? (this is `runnable` in the example above, as well any additional parameters you pass to `RunnableWithMessageHistory` to align the inputs/outputs)

Let's walk through these pieces (and more) below.

## How to store and load messages

A key part of this is storing and loading messages.
When constructing `RunnableWithMessageHistory` you need to pass in a `get_session_history` function.
This function should take in a `session_id` and return a `BaseChatMessageHistory` object.

**What is `session_id`?** 

`session_id` is an identifier for the session (conversation) thread that these input messages correspond to. This allows you to maintain several conversations/threads with the same chain at the same time.

**What is `BaseChatMessageHistory`?** 

`BaseChatMessageHistory` is a class that can load and save message objects. It will be called by `RunnableWithMessageHistory` to do exactly that. These classes are usually initialized with a session id.

Let's create a `get_session_history` object to use for this example. To keep things simple, we will use a simple SQLiteMessage

In [1]:
! rm memory.db

rm: memory.db: No such file or directory


In [2]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

Check out the [memory integrations](https://integrations.langchain.com/memory) page for implementations of chat message histories using other providers (Redis, Postgres, etc).

## What is the runnable you are trying to wrap?

`RunnableWithMessageHistory` can only wrap certain types of Runnables. Specifically, it can be used for any Runnable that takes as input one of:

* a sequence of [`BaseMessages`](/docs/concepts/#message-types)
* a dict with a key that takes a sequence of `BaseMessages`
* a dict with a key that takes the latest message(s) as a string or sequence of `BaseMessages`, and a separate key that takes historical messages

And returns as output one of

* a string that can be treated as the contents of an `AIMessage`
* a sequence of `BaseMessage`
* a dict with a key that contains a sequence of `BaseMessage`

Let's take a look at some examples to see how it works. 

### Setup

First we construct a runnable (which here accepts a dict as input and returns a message as output):

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>
```

In [3]:
# | output: false
# | echo: false

# %pip install -qU langchain langchain_anthropic

# import os
# from getpass import getpass

# os.environ["ANTHROPIC_API_KEY"] = getpass()
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)

In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory

### Messages input, message(s) output

The simplest form is just adding memory to a ChatModel.
ChatModels accept a list of messages as input and output a message.
This makes it very easy to use `RunnableWithMessageHistory` - no additional configuration is needed!

In [5]:
runnable_with_history = RunnableWithMessageHistory(
    model,
    get_session_history,
)

In [6]:
runnable_with_history.invoke(
    [HumanMessage(content="hi - im bob!")],
    config={"configurable": {"session_id": "1"}},
)

/Users/bagatur/langchain/.venv/lib/python3.11/site-packages/langchain_core/runnables/history.py:565: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


AIMessage(content="It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. How can I help you today?", additional_kwargs={}, response_metadata={'id': 'msg_01PxzQcbgQ55CBLV2Z7TsbKi', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 32}}, id='run-0bcb061e-4090-4297-8ee8-23fc1fb166ee-0', usage_metadata={'input_tokens': 12, 'output_tokens': 32, 'total_tokens': 44})

In [7]:
runnable_with_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "1"}},
)

AIMessage(content='I\'m afraid I don\'t actually know your name - you introduced yourself as Bob, but I don\'t have any other information about your identity. As an AI assistant, I don\'t have a way to independently verify people\'s names or identities. I\'m happy to continue our conversation, but I can only refer to you as "Bob" based on the introduction you provided.', additional_kwargs={}, response_metadata={'id': 'msg_01QQ3voLWGWSKr8SQpdP6fSX', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 52, 'output_tokens': 79}}, id='run-4ff28ac4-56b3-49c6-b757-77fb890fc3ff-0', usage_metadata={'input_tokens': 52, 'output_tokens': 79, 'total_tokens': 131})

:::info

Note that in this case the context is preserved via the chat history for the provided `session_id`, so the model knows the users name.

:::

We can now try this with a new session id and see that it does not remember.

In [8]:
runnable_with_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "1a"}},
)

AIMessage(content="I'm afraid I don't actually know your name. As an AI assistant, I don't have personal information about you unless you provide it to me directly.", additional_kwargs={}, response_metadata={'id': 'msg_01EzGiMbCGDBLxAkTPmhWKwt', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 35}}, id='run-f87f6c14-d697-4385-a067-787e777ca846-0', usage_metadata={'input_tokens': 12, 'output_tokens': 35, 'total_tokens': 47})

:::info      

When we pass a different `session_id`, we start a new chat history, so the model does not know what the user's name is. 

:::

### Dictionary input, message(s) output

Besides just wrapping a raw model, the next step up is wrapping a prompt + LLM. This now changes the input to be a **dictionary** (because the input to a prompt is a dictionary). This adds two bits of complication.

First: a dictionary can have multiple keys, but we only want to save ONE as input. In order to do this, we now now need to specify a key to save as the input.

Second: once we load the messages, we need to know how to save them to the dictionary. That equates to know which key in the dictionary to save them in. Therefore, we need to specify a key to save the loaded messages in.

Putting it all together, that ends up looking something like:

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who speaks in {language}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | model

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

:::info

Note that we've specified `input_messages_key` (the key to be treated as the latest input message) and `history_messages_key` (the key to add historical messages to).

:::

In [10]:
runnable_with_history.invoke(
    {"language": "italian", "input": "hi im bob!"},
    config={"configurable": {"session_id": "2"}},
)

AIMessage(content='Ciao Bob! È un piacere conoscerti. Come stai oggi?', additional_kwargs={}, response_metadata={'id': 'msg_01LGeTbfceQigNeJeAcRzJ38', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 29, 'output_tokens': 22}}, id='run-406ae666-fcb4-4bb3-b4fa-066868e4cd08-0', usage_metadata={'input_tokens': 29, 'output_tokens': 22, 'total_tokens': 51})

In [11]:
runnable_with_history.invoke(
    {"language": "italian", "input": "whats my name?"},
    config={"configurable": {"session_id": "2"}},
)

AIMessage(content='Bob, come hai detto prima.', additional_kwargs={}, response_metadata={'id': 'msg_01HCdvanvfZwJt2admvUqQ6f', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 60, 'output_tokens': 11}}, id='run-87154be2-9220-43b2-a4c6-555d0ecb2fba-0', usage_metadata={'input_tokens': 60, 'output_tokens': 11, 'total_tokens': 71})

:::info

Note that in this case the context is preserved via the chat history for the provided `session_id`, so the model knows the users name.

:::

We can now try this with a new session id and see that it does not remember.

In [12]:
runnable_with_history.invoke(
    {"language": "italian", "input": "whats my name?"},
    config={"configurable": {"session_id": "2a"}},
)

AIMessage(content='Mi dispiace, non so il tuo nome. Come posso aiutarti?', additional_kwargs={}, response_metadata={'id': 'msg_01158MsvHMQhZqvw1SyWJGpS', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 30, 'output_tokens': 23}}, id='run-1764141f-86d6-48b8-b53a-7c602c3fa50f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 23, 'total_tokens': 53})

:::info      

When we pass a different `session_id`, we start a new chat history, so the model does not know what the user's name is. 

:::

### Messages input, dict output

This format is useful when you are using a model to generate one key in a dictionary.

In [13]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": model})


runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

:::info

Note that we've specified `output_messages_key` (the key to be treated as the output to save).

:::

In [14]:
runnable_with_history.invoke(
    [HumanMessage(content="hi - im bob!")],
    config={"configurable": {"session_id": "3"}},
)

{'output_message': AIMessage(content="It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. How can I help you today?", additional_kwargs={}, response_metadata={'id': 'msg_017BLwqFNLtAumkKBPiEMFB3', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 32}}, id='run-c8811205-5bfe-4c56-9f76-0375ed932a93-0', usage_metadata={'input_tokens': 12, 'output_tokens': 32, 'total_tokens': 44})}

In [15]:
runnable_with_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "3"}},
)

{'output_message': AIMessage(content='I\'m afraid I don\'t actually know your name - you introduced yourself as Bob, but I don\'t have any other information about your identity. As an AI assistant, I don\'t have a way to independently verify people\'s names or identities. I\'m happy to continue our conversation, but I can only refer to you as "Bob" based on the introduction you provided.', additional_kwargs={}, response_metadata={'id': 'msg_01APvARxq24ucrWjnqRU9jt1', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 52, 'output_tokens': 79}}, id='run-f7735dff-76ac-4930-8e89-99d7f935319b-0', usage_metadata={'input_tokens': 52, 'output_tokens': 79, 'total_tokens': 131})}

:::info

Note that in this case the context is preserved via the chat history for the provided `session_id`, so the model knows the users name.

:::

We can now try this with a new session id and see that it does not remember.

In [16]:
runnable_with_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "3a"}},
)

{'output_message': AIMessage(content="I'm afraid I don't actually know your name. As an AI assistant, I don't have personal information about you unless you provide it to me directly.", additional_kwargs={}, response_metadata={'id': 'msg_01TtguudqCqct4N76qKXtcev', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 35}}, id='run-2f6393ac-0efd-4459-9ad0-9c849e07aabd-0', usage_metadata={'input_tokens': 12, 'output_tokens': 35, 'total_tokens': 47})}

:::info      

When we pass a different `session_id`, we start a new chat history, so the model does not know what the user's name is. 

:::

### Dict with single key for all messages input, messages output

This is a specific case of "Dictionary input, message(s) output". In this situation, because there is only a single key we don't need to specify as much - we only need to specify the `input_messages_key`.

In [17]:
from operator import itemgetter

runnable_with_history = RunnableWithMessageHistory(
    itemgetter("input_messages") | model,
    get_session_history,
    input_messages_key="input_messages",
)

:::info

Note that we've specified `input_messages_key` (the key to be treated as the latest input message).

:::

In [18]:
runnable_with_history.invoke(
    {"input_messages": [HumanMessage(content="hi - im bob!")]},
    config={"configurable": {"session_id": "4"}},
)

AIMessage(content="It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. How can I help you today?", additional_kwargs={}, response_metadata={'id': 'msg_01Gqb76JyziFdzju1pBgxFgg', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 32}}, id='run-722ca3c9-06f2-4984-b81d-32b16824ac6b-0', usage_metadata={'input_tokens': 12, 'output_tokens': 32, 'total_tokens': 44})

In [19]:
runnable_with_history.invoke(
    {"input_messages": [HumanMessage(content="whats my name?")]},
    config={"configurable": {"session_id": "4"}},
)

AIMessage(content='I\'m afraid I don\'t actually know your name - you introduced yourself as Bob, but I don\'t have any other information about your identity. As an AI assistant, I don\'t have a way to independently verify people\'s names or identities. I\'m happy to continue our conversation, but I can only refer to you as "Bob" based on the introduction you provided.', additional_kwargs={}, response_metadata={'id': 'msg_01CU5ZuJKFddF6YQeWQY2rzn', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 52, 'output_tokens': 79}}, id='run-2be5b746-774b-48e9-877e-622756205dd5-0', usage_metadata={'input_tokens': 52, 'output_tokens': 79, 'total_tokens': 131})

:::info

Note that in this case the context is preserved via the chat history for the provided `session_id`, so the model knows the users name.

:::

We can now try this with a new session id and see that it does not remember.

In [20]:
runnable_with_history.invoke(
    {"input_messages": [HumanMessage(content="whats my name?")]},
    config={"configurable": {"session_id": "4a"}},
)

AIMessage(content="I'm afraid I don't actually know your name. As an AI assistant, I don't have personal information about you unless you provide it to me directly.", additional_kwargs={}, response_metadata={'id': 'msg_01C6euAD4fPgWR8gA5aAQM6p', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 35}}, id='run-b264319f-3be6-4436-9b9c-4204a9c2766a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 35, 'total_tokens': 47})

:::info      

When we pass a different `session_id`, we start a new chat history, so the model does not know what the user's name is. 

:::

## Customization

The configuration parameters by which we track message histories can be customized by passing in a list of ``ConfigurableFieldSpec`` objects to the ``history_factory_config`` parameter. Below, we use two parameters: a `user_id` and `conversation_id`.

In [21]:
from langchain_core.runnables import ConfigurableFieldSpec


def get_session_history(user_id: str, conversation_id: str):
    return SQLChatMessageHistory(f"{user_id}--{conversation_id}", "sqlite:///memory.db")


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

with_message_history.invoke(
    {"language": "italian", "input": "hi im bob!"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

AIMessage(content='Ciao Bob! È un piacere conoscerti. Come stai oggi?', additional_kwargs={}, response_metadata={'id': 'msg_01KPra6oyE9KitL9pLhVURxX', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 29, 'output_tokens': 22}}, id='run-05c9ff13-6dcd-4201-aec9-88457a8469af-0', usage_metadata={'input_tokens': 29, 'output_tokens': 22, 'total_tokens': 51})

In [22]:
# remembers
with_message_history.invoke(
    {"language": "italian", "input": "whats my name?"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

AIMessage(content='Bob, come hai detto prima.', additional_kwargs={}, response_metadata={'id': 'msg_01G4zGQvS9APVq7BiyScAFoz', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 60, 'output_tokens': 11}}, id='run-feaf114d-5e2f-41be-8274-182db7ddead7-0', usage_metadata={'input_tokens': 60, 'output_tokens': 11, 'total_tokens': 71})

In [23]:
# New user_id --> does not remember
with_message_history.invoke(
    {"language": "italian", "input": "whats my name?"},
    config={"configurable": {"user_id": "456", "conversation_id": "1"}},
)

AIMessage(content='Mi dispiace, non so il tuo nome. Come posso aiutarti?', additional_kwargs={}, response_metadata={'id': 'msg_012ydNSs4QUyH4dNmzX4soJZ', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 30, 'output_tokens': 23}}, id='run-aaa1a636-cf61-493d-a95b-1c09442728e8-0', usage_metadata={'input_tokens': 30, 'output_tokens': 23, 'total_tokens': 53})

Note that in this case the context was preserved for the same `user_id`, but once we changed it, the new chat history was started, even though the `conversation_id` was the same.